In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import jax
from jax import lax
from deluca.envs import Acrobot
from deluca.agents import ILQR


In [147]:
agent = ILQR()
agent.train(Acrobot(horizon=10), 10)

iLQR : t = 2, c = 0.008625
iLQR : t = 3, c = 0.008625
iLQR : t = 4, c = 0.008625
iLQR : t = 5, c = 0.008625
iLQR : t = 6, c = 0.008625
iLQR : t = 7, c = 0.008625
iLQR : t = 8, c = 0.008625
iLQR : t = 9, c = 0.008625
iLQR : t = 10, c = 0.008625


In [148]:
def loop(context, x):
    env, agent = context
    control = agent(env.state)
    _, reward, _, _ = env.step(control)
    return (env, agent), reward

In [149]:
 # for loop version
T = 10
env = Acrobot(horizon=10)
print(env.reset())
print('agent.t = ' + str(agent.t))
reward = 0
plan = agent.U
print(plan)
for i in range(T):
    (env, agent), r = loop((env, agent), 0)
    reward += r
reward_forloop = reward
print('reward_forloop = ' + str(reward_forloop))

[-0.08660281 -0.00455228  0.00775968 -0.01845726]
agent.t = 0
[[ 0.00829666]
 [ 0.0103582 ]
 [ 0.00891292]
 [ 0.00522574]
 [ 0.00112483]
 [-0.00178463]
 [-0.00272972]
 [-0.00199128]
 [-0.00065128]
 [ 0.        ]]
reward_forloop = 0.021269226351404824


In [151]:
# scan version
T = 10
env = Acrobot(horizon=10)
print(env.reset())
agent.reset()
print('agent.t = ' + str(agent.t))
xs = jnp.array(jnp.arange(T))
_,reward_scan = lax.scan(loop, (env, agent), xs)
# correctness test
print('reward_scan sum = ' + str(jnp.sum(reward_scan)))

[-0.08660281 -0.00455228  0.00775968 -0.01845726]
agent.t = 0
reward_scan sum = 0.021269226351404935
